# FFmpeg example: encoding

In [1]:
path = %env PATH
path = path + ":./ffmpeg-4.2.2-amd64-static"
%env PATH $path

!which ffmpeg

env: PATH=/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin:./ffmpeg-4.2.2-amd64-static
/usr/local/bin/ffmpeg


In [38]:
# Print input video information.
!ffprobe input.mp4

ffprobe version 4.2.2 Copyright (c) 2007-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2_3 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags=-fno-stack-check --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.

In [39]:
# Compress input.mp4 to target bitrate 1000kbps and save as output.mp4.

!ffmpeg -hide_banner -i input.mp4 -b:v 1000k -y output.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288 tbn, 48 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x7ff3ce00ac00] using SAR=1/1
[libx264 @ 0x7ff3ce00ac00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7ff3ce00ac00] profile High, level 3.0
[libx264 @ 0x7ff3ce00ac00] 264 - core 155

In [2]:
# Calculate VMAF between input.mp4 (ref) and output.mp4 (target).

!ffmpeg -hide_banner -i output.mp4 -i input.mp4 \
-lavfi "libvmaf=model_path=ffmpeg-4.2.2-amd64-static/model/vmaf_v0.6.1.pkl" -f null -

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288 tbn, 48 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Input #1, mov,mp4,m4a,3gp,3g2,mj2, from 'output.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.03, start: 0.000000, bitrate: 1176 kb/s
    Stream #1:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1041 kb/s, 24 fps, 24 tbr, 12288

In [5]:
# Calculate PSNR between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi psnr -f null - 2>&1 | grep "Parsed_psnr_0"

[Parsed_psnr_0 @ 0x7ff72257b580] PSNR y:41.397824 u:46.246450 v:47.694155 average:42.587721 min:38.395619 max:54.457783


In [7]:
# Calculate SSIM between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi sim -f null - 2>&1 

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'output.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.03, start: 0.000000, bitrate: 1176 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1041 kb/s, 24 fps, 24 tbr, 12288 tbn, 48 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Input #1, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
    Stream #1:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288